## 유럽의 직장 환경(만족감, 효능감)에 따른 분류 
- https://www.kaggle.com/guruarun93/european-working-conditions-survey-2016

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 데이터 입력(7813x11, 11개의 attribute)
df = pd.read_csv('./data/EWCS_2016.csv')

x = df.iloc[:,2:].copy()  # Q2a(성별), Q2b(연령)는 제외

for col in x.columns:
    x.drop(x.index[x[col] == -999].tolist(), inplace = True)  # 미입력값이 있는 index 제거(7659x9)

x1 = x.loc[:,"Q87a":"Q87e"]  # 만족감 설문 결과
x2 = x.loc[:,"Q90a":"Q90f"]  # 효능감 설문 결과
K = 4; N = x.shape[0]; C = 2  # 3개의 군집, 2차원으로 특징추출

### 특징 추출(PCA)

In [3]:
#PCA를 사용한 차원 감소를 통한 특징 추출
from sklearn.decomposition import PCA

pca = PCA(n_components=1)  # 1차원 벡터로 사영
x1 =pca.fit_transform(x1)  # 만족감 특징 추출
x2 =pca.fit_transform(x2)  # 효능감 특징 추출
x_2d = np.hstack([x1, x2]) 

### K-means Clustering

In [ ]:
# 임의의 대표 벡터 설정
mean = np.array([x1.mean(), x2.mean()])
std = np.array([x1.std(), x2.std()])
centers = np.random.randn(K, C)*std+mean 

plt.scatter(x1, x2, c='black', s=7, label = 'Data')
plt.scatter(centers[:,0], centers[:,1], marker='s', s = 30, c='m', label ='Center')

In [ ]:
import copy

# 초기화
centers_old = np.zeros((K, C))   
centers_new = copy.deepcopy(centers)    
clusters = np.zeros(N)             
distances = np.zeros((N,K))

In [ ]:
# K-meaans 반복 수정
MAXiter = 10  # 최대 반복 횟수
color = ['red','blue','green','black']
for iter in range(MAXiter):
    for i in range(K):
        distances[:,i] = np.linalg.norm(x_2d - centers_new[i], axis=1) # 중심 벡터와의 거리 계산

    clusters = np.argmin(distances, axis=1)  # 가장 가까운 군집에 포함
    centers_old = copy.deepcopy(centers_new)  # 중심 벡터 최신화 

    for i in range(K):
        centers_new[i] = np.mean(x_2d[clusters==i], axis=0)  # 군집에 포함된 데이터들의 평균 벡터를 사용하여 수정
        plt.scatter(x_2d[clusters==i,0], x_2d[clusters==i,1], s=7, c=color[i])
        plt.scatter(centers_new[i,0], centers_new[i,1], marker='s', s=50, c='m')

    plt.title('K-means (iter={})'.format(iter))
    plt.xlabel('X1'); plt.ylabel('Y1'); 
    plt.show()
    print('Iteration = {}'.format(iter))
    print(centers_new)


In [ ]:
# 군집에 포함된 데이터 수 출력
unique, counts = np.unique(clusters, return_counts=True)

print(np.asarray((unique, counts)).T)